In [1]:
from google.colab import files
uploaded = files.upload()
!pip install python-dotenv

Saving .env to .env


Install Required Libraries

In [2]:
# Install required libraries
!pip install pandas requests
!pip uninstall supabase
!pip install supabase




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00


Import the Libraries

In [3]:
# Import necessary libraries
import pandas as pd
import requests
import uuid
from datetime import datetime
import os
from google.colab import drive
from dotenv import load_dotenv
from supabase import create_client, Client, ClientOptions




Step 1 Load Nursing Data File and Create the Users using the email Address. Also parse the sms chat text

In [4]:
# Load .env file
load_dotenv('.env')

# Access environment variables
SUPABASE_CANDIDATE_URL = os.getenv('SUPABASE_CANDIDATE_URL')
SUPABASE_PROJECT_URL = os.getenv('SUPABASE_PROJECT_URL')
SUPABASE_API_KEY = os.getenv('SUPABASE_API_KEY')
SUPABASE_SERVICE_KEY = os.getenv('SUPABASE_SERVICE_KEY')

supabase: Client = create_client(SUPABASE_PROJECT_URL, SUPABASE_SERVICE_KEY)

# File path to the uploaded CSV file
file_path = '/content/sample_data/Nursing -Helius Benchmark New.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)
df = df.dropna(subset=['Email']).reset_index(drop=True)





In [5]:
from supabase import create_client
from supabase.lib.client_options import ClientOptions

supabase = create_client(
  SUPABASE_PROJECT_URL,
  SUPABASE_SERVICE_KEY,
  options=ClientOptions(
    auto_refresh_token=False,
    persist_session=False,
  )
)

# Access auth admin api
admin_auth_client = supabase.auth.admin
user_list = supabase.auth.admin.list_users(1, 4000)


In [6]:
def get_user_id_by_email(email: str):
    try:
        for user in user_list:
            if user.email == email:
              return user.id
        return None
    except Exception as e:
        print(f"Error fetching user by email: {e}")
        return None

In [7]:
def check_user_exists(email: str):
    try:
        for user in user_list:
            if user.email == email:
              return True
        return False
    except Exception as e:
        print(f"Error fetching user by email: {e}")
        return False

In [16]:
def get_assessment_id_by_candidate(candidate_id):
    """
    Retrieves the assessment_id from the hel_assessment table given a candidate_id.

    Args:
        candidate_id (str): The UUID of the candidate.

    Returns:
        str: The assessment_id if found, otherwise None.
    """
    try:
        response = (
            supabase.table("hel_assessment")
            .select("assessment_id")
            .eq("candidate_id", candidate_id)
            .limit(1)
            .execute()
        )

        if response.data:
            return response.data[0]["assessment_id"]  # Return the first matching assessment_id
        else:
            print(f"No assessment found for candidate_id: {candidate_id}")
            return None

    except Exception as e:
        print(f"Error fetching assessment_id: {e}")
        return None


Helper Functions

In [8]:
def format_phone_number(phone_number):
    # Ensure the phone number is an integer and then convert to string
    phone_number = str(int(phone_number))

    # Format the phone number
    formatted_number = f"({phone_number[1:4]}) {phone_number[4:7]}-{phone_number[7:]}"
    return formatted_number

First create the User in the Auth Table

In [9]:
# Headers for admin access
headers = {
    "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
    "apikey": SUPABASE_SERVICE_KEY,
    "Content-Type": "application/json",
}

# Function to create or update (upsert) a user
def upsert_user(email, password):
    existing_user = check_user_exists(email)
    if existing_user:
        # Update logic (optional; Supabase admin API doesn't allow updates directly)
        print(f"User already exists: {email}. Skipping creation.")
    else:
        # Create new user
        data = {
            "email": email,
            "password": password,
            "email_confirm": True  # Automatically confirm the email
        }
        response = requests.post(
            f"{SUPABASE_PROJECT_URL}/auth/v1/admin/users", headers=headers, json=data
        )
        if response.status_code in [200, 201]:
            print(f"User created successfully: {email}")
        else:
            print(f"Failed to create user {email}: {response.text}")


In [ ]:
# Iterate through the DataFrame and upsert users
for index, row in df.iterrows():
    upsert_user(row["Email"], "test1234")

User already exists: drehanna.smith@gmail.com. Skipping creation.
User already exists: ajufof@yahoo.com. Skipping creation.
User already exists: florn21@aol.com. Skipping creation.
User already exists: ria1201@live.com. Skipping creation.
User already exists: unegbuonyinye@yahoo.com. Skipping creation.
User already exists: onyinyeunegbu@yahoo.com. Skipping creation.
User already exists: ravenpatton23@gmail.com. Skipping creation.
User already exists: shivashis.roy@gmail.com. Skipping creation.
User already exists: verasvenus@gmail.con. Skipping creation.
User already exists: kedmond.hanagreaterny@gmail.com. Skipping creation.
User already exists: laurenparler413@gmail.com. Skipping creation.
User already exists: edominique523@gmail.com. Skipping creation.
User already exists: dianah38@nycstudents.net. Skipping creation.
User already exists: isiahst.wolf@gmail.com. Skipping creation.
User already exists: galouchekap@nycstudents.net. Skipping creation.
User already exists: jepsiel@nycstu

In [10]:
# Function to split full_name into first_name and last_name
def split_full_name(full_name):
    name_parts = full_name.strip().split()
    first_name = name_parts[0] if name_parts else ''
    last_name = ' '.join(name_parts[1:]) if len(name_parts) > 1 else ''
    return first_name, last_name

# Function to determine process_stage
def determine_process_stage(assessment_date):
    if pd.notnull(assessment_date):
        return 'Assessment Completed'
    return 'User'

# Apply process_stage determination to the DataFrame
df['process_stage'] = df['Assessment Date'].apply(determine_process_stage)

# Add columns for first_name and last_name by splitting full_name
df[['first_name', 'last_name']] = df['Name'].apply(
    lambda full_name: pd.Series(split_full_name(full_name))
)



# Function to upload data to Supabase
def upload_to_supabase(row):
    headers = {
        "apikey": SUPABASE_SERVICE_KEY,
        "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
        "Content-Type": "application/json",
        "Prefer": "resolution=merge-duplicates"
    }
    # Map CSV data to the database columns
    data = {
        "candidate_id": get_user_id_by_email(row['Email']),  # Generate a new UUID for each candidate
        "first_name": row['first_name'],   # Use first_name from split
        "last_name": row['last_name'],     # Use last_name from split
        "email": row['Email'],
        "application_date": row['Application Date'],
        "status": row['Status'],
        "process_stage": row['process_stage'],
        "creation_date": datetime.now().date().isoformat(),  # Use current date
        "created_by": "ADMIN",
        "last_update_date": datetime.now().date().isoformat(),
        "last_updated_by": "ADMIN"
    }
    print(data)

    response = requests.post(SUPABASE_CANDIDATE_URL, headers=headers, json=data)
    if response.status_code in [200, 201]:
        print(f"Successfully uploaded: {row['Email']}")
    else:
        print(f"Failed to upload: {row['Email']} - {response.text}")

# Iterate through the DataFrame and upload each row
df.apply(upload_to_supabase, axis=1)

print("Data upload complete.")



{'candidate_id': '987ee143-7371-4851-b9e1-5969c40f55e3', 'first_name': 'Dre', 'last_name': 'Smith', 'email': 'drehanna.smith@gmail.com', 'application_date': '5-Feb-23', 'status': 'Prospect', 'process_stage': 'Assessment Completed', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: drehanna.smith@gmail.com
{'candidate_id': '78c5970d-7a12-4f57-8e19-d5732435612a', 'first_name': 'Faith', 'last_name': 'Ajufo.', 'email': 'ajufof@yahoo.com', 'application_date': '8-Mar-23', 'status': 'Prospect', 'process_stage': 'Assessment Completed', 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: ajufof@yahoo.com
{'candidate_id': '234abd72-e23a-4580-a2aa-b79b7efd658b', 'first_name': 'Florence', 'last_name': 'Pierre', 'email': 'florn21@aol.com', 'application_date': '16-Mar-23', 'status': 'Prospect', 'process_stage': 'Assessment Com

Populate Assessment Table

In [15]:
# Upsert data into the Assessment Table if an id was created
df_cleaned = df.dropna(subset=['Assessment Date'])
for index, row in df_cleaned.iterrows():
    email = row['Email']
    assessment_date = row['Assessment Date']
    assessment_kind = row['Assessment Kind']
    assessment_duration = row['Assessment Duration']
    fit_score = row['Fit Score']
    score = row['Score']

    if get_user_id_by_email(email) is not None:
      data = {
          "candidate_id": get_user_id_by_email(email),  # Generate a new UUID for each candidate
          "assessment_date": assessment_date,   # Assessment Date
          "assessment_kind": assessment_kind,   # Assessment Kind
          "assessment_duration": assessment_duration,   # Assessment Duration
          "fit_score": fit_score,   # Fit Score
          "score": score,   # Score
          "creation_date": datetime.now().date().isoformat(),
          "created_by": "ADMIN",
          "last_update_date": datetime.now().date().isoformat(),
          "last_updated_by": "ADMIN"
      }
      print(data)

      response = (
              supabase.table("hel_assessment")
               .upsert(data)
               .execute()
           )

      # Check if there is an error in the response
      if response:
          print(f"Successfully uploaded: {data['candidate_id']}")
      else:
          print(f"Failed to upload: {data['candidate_id']} - {response.error}")



{'candidate_id': '987ee143-7371-4851-b9e1-5969c40f55e3', 'assessment_date': '5-Feb-23', 'assessment_kind': 'express', 'assessment_duration': '0:18:48', 'fit_score': '6 - Good', 'score': 61.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 987ee143-7371-4851-b9e1-5969c40f55e3
{'candidate_id': '78c5970d-7a12-4f57-8e19-d5732435612a', 'assessment_date': '8-Mar-23', 'assessment_kind': 'express', 'assessment_duration': '0:24:23', 'fit_score': '6 - Good', 'score': 61.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 78c5970d-7a12-4f57-8e19-d5732435612a
{'candidate_id': '234abd72-e23a-4580-a2aa-b79b7efd658b', 'assessment_date': '16-Mar-23', 'assessment_kind': 'express', 'assessment_duration': '0:24:23', 'fit_score': '3 - Below average', 'score': 48.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_up

Populate the Scores Table

In [27]:
for index, row in df_cleaned.iterrows():
    email = row['Email']

    if get_user_id_by_email(email) is not None:
      data = {
          "assessment_id": get_assessment_id_by_candidate(get_user_id_by_email(email)),  # Generate a new UUID for each candidate
          "cognitive_fit_score" : row['Cognitive Fit Score'],
          "cognitive_match" : row['Cognitive Match'],
          "personality_fit_score" : row['Personality Fit Score'],
          "personality_match" : row['Personality Match'],
          "attention_to_detail_score" : row['Attention to Detail'],
          "attention_to_detail_weighed" : row['Attention to Detail Weighed'],
          "logical_reasoning_score" : row['Logical Reasoning'],
          "logical_reasoning_weighed" : row['Logical Reasoning Weighed'],
          "numerical_reasoning_score" : row['Numerical Reasoning'],
          "numerical_reasoning_weighed" : row['Numerical Reasoning Weighed'],
          "verbal_reasoning_score" : row['Verbal Reasoning'],
          "verbal_reasoning_weighed" : row['Verbal Reasoning Weighed'],
          "abstractedness_score" : row['Abstractedness'],
          "apprehension_score" : row['Apprehension'],
          "apprehension_match" : row['Apprehension Match'],
          "dominance_score" : row['Dominance'],
          "emotional_stability_score" : row['Emotional Stability'],
          "emotional_stability_match" : row['Emotional Stability Match'],
          "liveliness_score" : row['Liveliness'],
          "liveliness_match" : row['Liveliness Match'],
          "openness_to_change_score" : row['Openness to Change'],
          "openness_to_change_match" : row['Openness to Change Match'],
          "perfectionism_score" : row['Perfectionism'],
          "perfectionism_match" : row['Perfectionism Match'],
          "privateness_score" : row['Privateness'],
          "reasoning_score" : row['Reasoning'],
          "reasoning_match" : row['Reasoning Match'],
          "rule_consciousness_score" : row['Rule-Consciousness'],
          "self_reliance_score" : row['Self-Reliance'],
          "self_reliance_match" : row['Self-Reliance Match'],
          "sensitivity_score" : row['Sensitivity'],
          "social_boldness_score" : row['Social Boldness'],
          "social_boldness_match" : row['Social Boldness Match'],
          "tension_score" : row['Tension'],
          "vigilance_score" : row['Vigilance'],
          "vigilance_match" : row['Vigilance Match'],
          "warmth_score" : row['Warmth'],
          "warmth_match" : row['Warmth Match'],
          "sales_and_customer_service" : row['Sales and Customer Service (deprecated)'],
          "problem_solving" : row['Problem Solving / Complex Thinking'],
          "url" : row['URL'],
          "creation_date": datetime.now().date().isoformat(),
          "created_by": "ADMIN",
          "last_update_date": datetime.now().date().isoformat(),
          "last_updated_by": "ADMIN"
      }


      print(data)

      response = (
              supabase.table("hel_scores")
               .upsert(data)
               .execute()
           )

      # Check if there is an error in the response
      if response:
          print(f"Successfully uploaded: {data['assessment_id']}")
      else:
          print(f"Failed to upload: {data['assessment_id']} - {response.error}")



{'assessment_id': 'ab70c750-d836-4255-a1e3-ef5211dc0a88', 'cognitive_fit_score': '6 - Good', 'cognitive_match': 65.0, 'personality_fit_score': '6 - Good', 'personality_match': 57.0, 'attention_to_detail_score': 100.0, 'attention_to_detail_weighed': 25.0, 'logical_reasoning_score': 60.0, 'logical_reasoning_weighed': 15.0, 'numerical_reasoning_score': 40.0, 'numerical_reasoning_weighed': 10.0, 'verbal_reasoning_score': 60.0, 'verbal_reasoning_weighed': 15.0, 'abstractedness_score': 67.0, 'apprehension_score': 25.0, 'apprehension_match': 75.0, 'dominance_score': 50.0, 'emotional_stability_score': 75.0, 'emotional_stability_match': 75.0, 'liveliness_score': 58.0, 'liveliness_match': 42.0, 'openness_to_change_score': 42.0, 'openness_to_change_match': 42.0, 'perfectionism_score': 33.0, 'perfectionism_match': 67.0, 'privateness_score': 50.0, 'reasoning_score': 83.0, 'reasoning_match': 17.0, 'rule_consciousness_score': 58.0, 'self_reliance_score': 25.0, 'self_reliance_match': 75.0, 'sensitivit

Populate the Traits Table

In [29]:
for index, row in df_cleaned.iterrows():
    email = row['Email']

    if get_user_id_by_email(email) is not None:
      data = {
          "assessment_id": get_assessment_id_by_candidate(get_user_id_by_email(email)),  # Generate a new UUID for each candidate
          "openness" : row['Big 5 (OCEAN): Openness'],
          "conscientiousness" : row['Big 5 (OCEAN): Conscientiousness'],
          "extraversion" : row['Big 5 (OCEAN): Extraversion'],
          "agreeableness" : row['Big 5 (OCEAN): Agreeableness'],
          "neuroticism" : row['Big 5 (OCEAN): Neuroticism'],
          "creation_date": datetime.now().date().isoformat(),
          "created_by": "ADMIN",
          "last_update_date": datetime.now().date().isoformat(),
          "last_updated_by": "ADMIN"
      }

      print(data)

      response = (
              supabase.table("hel_traits")
               .upsert(data)
               .execute()
           )

      # Check if there is an error in the response
      if response:
          print(f"Successfully uploaded: {data['assessment_id']}")
      else:
          print(f"Failed to upload: {data['assessment_id']} - {response.error}")



{'assessment_id': 'ab70c750-d836-4255-a1e3-ef5211dc0a88', 'openness': 54.0, 'conscientiousness': 42.0, 'extraversion': 62.0, 'agreeableness': 52.0, 'neuroticism': 31.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: ab70c750-d836-4255-a1e3-ef5211dc0a88
{'assessment_id': 'e96d5a4c-62b5-4b69-b0e2-b4a56c68c492', 'openness': 58.0, 'conscientiousness': 59.0, 'extraversion': 80.0, 'agreeableness': 59.0, 'neuroticism': 0.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: e96d5a4c-62b5-4b69-b0e2-b4a56c68c492
{'assessment_id': '8d0ba4d4-76ad-4ab8-9f15-5caec53ef068', 'openness': 50.0, 'conscientiousness': 46.0, 'extraversion': 57.0, 'agreeableness': 42.0, 'neuroticism': 59.0, 'creation_date': '2025-02-02', 'created_by': 'ADMIN', 'last_update_date': '2025-02-02', 'last_updated_by': 'ADMIN'}
Successfully uploaded: 8d0

# New Section